##### Routes

Defines the FastAPI router, which is the HTTP layer that sits between incoming requests from the Svelte frontend and the backend modules. It contains the Pydantic request models for input validation and 3 route handler functions.

Four Pydantic models are defined to describe the expected shape of the POST/timegraph request body:

1. TimeFrameModel
2. ZoneBoundariesModel
3. FittingModel
4. TimegraphRequest 

FastAPI automatically validates any incoming request body against TimeGraphRequest before the handler function is called, and returns a 422 ERROR if validation fails.

Importing and defining stuff

In [ ]:
import os
from datetime import datetime, timezone
from uuid import uuid4

from fastapi import APIRouter, HTTPException, Request
from pydantic import BaseModel, Field

from core.state_store.archive_reader import read_timeseries
from core.analysis.trajectory import compute_trajectory
from core.state_store.database import get_connection
from core.output.report_serializer import save_timegraph_report


router = APIRouter()

Defining the Pydantic request models

In [ ]:
class TimeframeModel(BaseModel):
    start_time: str
    end_time: str

    model_config = {"populate_by_name": True}


class ZoneBoundariesModel(BaseModel):
    healthy_min: float
    healthy_max: float
    vulnerability_margin: float


class FittingModel(BaseModel):
    polynomial_degree: int


class TimegraphRequest(BaseModel):
    subject_id: str
    module_id: str
    marker_id: str
    timeframe: TimeframeModel
    zone_boundaries: ZoneBoundariesModel
    fitting: FittingModel

Routes

request.app.state is FastAPI's mechanism for storing application-wide data:

**GET /registry:** Returns the module registry that was loaded into the application state at server startup. In this case the resistry dict was placed there by main.py when the server first booted. This provides the frontend with the info it needs to populate fields

**GET /subjects:** Instead of querying the database for known subjects, this endpoint discovers subjects by inspecting the archive directory directly. os.listdir returns all names in a directory, and in this case filters out everything thats not itself a directory (all subjects are folders with their respective data).

**POST /timegraph:** This is the primary endpoint, and it triggers a 5-stage sequence:

    1. Parses the timeframe: The ISO 8601 timestamp strings from the request body are converted to timezone-aware python datetime objects. If parsing fails, returns 422 ERROR.

    2. Reads the archive: read_timeseries() is called with the path parameters and time boundaries. Returns an ERROR if the respective index file does not exist or does not contain any datapoints within the timeframe. 

    3. Run trajectory computation: The Pydantic model fields are unpacked into a plain dict matching the signature that compute_trajectory() expects. If there are insufficient data points to run the polynomial fitting algorithm it returns a 422 ERROR

    4. A UUID is generated to uniquely identify this report, along with the current UTC time (requested_at). This is passed (along with all input parameters and trajectory result) to save_timegraph_report, which writes a full JSON to disk and inserts the metadata into the database. 

    5. Register the subject and return: "INSERT OR IGNORE" attempts to add the subject to the "subjects" table of the database, but if it already exists it doesn;t do anything. It then returns the data needed for the frontend to render the chart.